## LeNet (18/06/08)

In [2]:
# 导入mnist的数据集
from tensorflow.examples.tutorials.mnist import input_data
# read_data_sets(train_dir,fake_data=False,one_hot=True,dtype=dtypes.float32,
                   #reshape=True, validation_size=5000,seed=None)
# 过程就是下载数据，解压数据，返回datasets，包括train，validation,test三个部分
# one_hot = True 是考虑多类情况。非onehot，标签是0 1 2 3 这样的
# one_hot 就是一个长度为 n 的数组，只有一个元素是1，其他元素是0
# mnist的标签就是这样的，所以one_hot = True
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
sess = tf.InteractiveSession()
def weight(shape):  
    initial = tf.truncated_normal(shape,stddev=0.1)  
    return tf.Variable(initial)  
  
def bias(shape):  
    initial = tf.constant(0.1,shape = shape)  
    return tf.Variable(initial)  
  
def conv2(x,W):  
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='VALID')  
  
def max_pool(x):  
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')  
  
x = tf.placeholder(tf.float32,[None,784])  
y_ = tf.placeholder(tf.float32,[None,10])  
x_image=tf.pad(tf.reshape(x,[-1,28,28,1]),[[0,0],[2,2],[2,2],[0,0]])  


In [4]:
# 第一层卷积 + 池化
# 6个5x5的卷积核
W_conv1 = weight([5,5,1,6])
b_conv1 = bias([6])
h_conv1 = tf.nn.relu(conv2(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1)

In [5]:
# 第二层卷积 + 池化
# 16个5x5的卷积核
W_conv2 = weight([5,5,6,16])
b_conv2 = bias([16])
h_conv2 = tf.nn.relu(conv2(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2)

In [6]:
# 第三层卷积
# 120个5x5的卷积核
W_conv3 = weight([5,5,16,120])
b_conv3 = bias([120])
h_conv3 = tf.nn.relu(conv2(h_pool2,W_conv3) + b_conv3)
h_conv3_re  = tf.reshape(h_conv3,[-1,120])

In [7]:

keep_prob = tf.placeholder(tf.float32)  
        
                   
# 全连接层
W_fc1 = weight([1*1*120,84])
b_fc1 = bias([84])
h_fc1 = tf.nn.relu(tf.matmul(h_conv3_re,W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# 输出层
W_o = weight([1*1*84,10])
b_o = bias([10])
y_predict = tf.nn.softmax(tf.matmul(h_fc1_drop,W_o) + b_o)


                   
                   

In [8]:

cross_entropy = tf.reduce_mean(  
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_predict)) 

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
# train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)  

# 算正确率
correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y_, 1))  

# 运行上面这个函数，会生成一组向量，如：[True, False, True, True]。
# 把它映射成浮点数，然后，计算它们的均值
# cast(x, dtype, name=None) ,将x的数据格式转化成dtype
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  

#开始训练
sess.run(tf.global_variables_initializer())  
for i in range(20000):  
    # 50一组 把mnist上的东西赋给了batch
    # 之后x = batch[0],y_ = batch[1]
    batch = mnist.train.next_batch(50) 
    
    if i % 1000 == 0:  
        train_accuracy = accuracy.eval(feed_dict={  
            x: batch[0], y_: batch[1], keep_prob: 1.0})  
        print("step %d, training accuracy %g" % (i, train_accuracy))  
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})  

print("step %d, training accuracy %g" % (i, train_accuracy))  

step 0, training accuracy 0.08
step 1000, training accuracy 0.86
step 2000, training accuracy 0.86
step 3000, training accuracy 0.9
step 4000, training accuracy 0.96
step 5000, training accuracy 0.92
step 6000, training accuracy 1
step 7000, training accuracy 0.96
step 8000, training accuracy 0.96
step 9000, training accuracy 0.96
step 10000, training accuracy 0.98
step 11000, training accuracy 0.94
step 12000, training accuracy 0.94
step 13000, training accuracy 0.94
step 14000, training accuracy 0.98
step 15000, training accuracy 1
step 16000, training accuracy 0.98
step 17000, training accuracy 0.96
step 18000, training accuracy 1
step 19000, training accuracy 0.98
step 19999, training accuracy 0.98
